In [24]:
from IPython.display import clear_output, Image, display, Markdown

from ipywidgets import interact, widgets, HBox, VBox, Layout, Box

from base64 import b64encode
import random
import cv2
import time
import requests
import boto3
import uuid
import json

#initialized resources
servicediscovery = boto3.client('servicediscovery')
personalize_runtime = boto3.client('personalize-runtime')
kds = boto3.client('kinesis')


rfy_recommender_arn = 'arn:aws:personalize:us-east-1:913089978341:recommender/octank-recommended-for-you'

wvx_recommender_arn = 'arn:aws:personalize:us-east-1:913089978341:recommender/octank-who-viewed-x-also-viewed'


product_image_url='http://d23ar2gbu3zcc4.cloudfront.net/images/{}/{}.jpg'

stream_name = 'octank-event-stream'

event_tracking_id = 'dc968701-e8f1-4a7c-8e39-730fa9957247'

video_id = 'movie-9053391.mp4'
video_path = f'videos/{video_id}'

## Connect to user and item microservice

In [3]:
# Pull user info
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='users',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Users service instance not found; check ECS to ensure it launched cleanly'

users_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Users Service Instance IP: {}'.format(users_service_instance))

Users Service Instance IP: 10.215.10.92


In [4]:
# Pull product info
response = servicediscovery.discover_instances(
    NamespaceName='retaildemostore.local',
    ServiceName='products',
    MaxResults=1,
    HealthStatus='HEALTHY'
)

assert len(response['Instances']) > 0, 'Products service instance not found; check ECS to ensure it launched cleanly'

products_service_instance = response['Instances'][0]['Attributes']['AWS_INSTANCE_IPV4']
print('Products Service Instance IP: {}'.format(products_service_instance))

Products Service Instance IP: 10.215.10.136


## Function to help visualize recommended items for this user

In [7]:
def evaluate_item_list(item_list, show=False):

    Vbox_list = []
    for item in item_list:
        response = requests.get('http://{}/products/id/{}'.format(products_service_instance, item['itemId']))
        item_details = response.json()
        
        product_image = item_details["image"]
        product_name = item_details["name"]
        product_category = item_details["category"]

        Vbox_list.append(VBox([widgets.Label(product_name),
                               widgets.Label(product_category),
                               widgets.Image(value=Image(product_image).data)]))
    
    hbox = HBox(Vbox_list)
    if show:
        display(hbox)
    
    return hbox
    
def recommended_item_layout(item_list, show=False):

    # We are going to provide 4 recommendations from the list
    left_vbox_list =[]
    right_vbox_list =[]
    for i in range(len(item_list)):
        response = requests.get('http://{}/products/id/{}'.format(products_service_instance, item_list[i]['itemId']))
        item_details = response.json()
        
        product_image = item_details["image"]
        
        if i%2 != 0:
            left_vbox_list.append(widgets.Image(value=Image(product_image).data))
        else:
            right_vbox_list.append(widgets.Image(value=Image(product_image).data))
        
    left_box = VBox(left_vbox_list)
    right_box = VBox(right_vbox_list)
    
    hbox = HBox([left_box, right_box])
    if show:
        display(hbox)
    return hbox

In [8]:
def get_recommendation(recommender_arn, k, user_id, item_id=None):
    
    get_recommendations_response = None
    
    if item_id is None:
        #Get recommendation off the first frame
        get_recommendations_response = personalize_runtime.get_recommendations(
            recommenderArn = recommender_arn,
            userId = user_id,
            numResults = k)
    else:
        #Get recommendation off the first frame
        get_recommendations_response = personalize_runtime.get_recommendations(
            recommenderArn = recommender_arn,
            userId = str(user_id),
            itemId = str(item_id),
            numResults = k)
        
    item_list = get_recommendations_response['itemList']
    return item_list

## Orginal Video

In [9]:
from IPython.display import display

cap = cv2.VideoCapture(video_path)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_id = 0

orig_video_frames = []

while True:
    _, frame = cap.read()
        
    frame_bytes = cv2.imencode('.jpeg', frame)[1].tobytes()

    orig_video_frames.append(frame_bytes)
    
    frame_id += 1
    if frame_id >= frame_count:
        break
        
def f(Frame_numb):
    i = Frame_numb-1
    image = orig_video_frames[i]
    
    video_frame = Box(children=[widgets.Image(value=image)])
    display(video_frame)

play = widgets.Play(
    value=1,
    min=1,
    max=frame_count,
    step=1,
    interval=2000,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider(min=1,max=frame_count,step=1,value=1)
widgets.jslink((play, 'value'), (slider, 'value'))
ui = widgets.HBox([play, slider])

output = widgets.interactive_output(f, {'Frame_numb': play})
display(ui, output)

Output()

## Demo Processed Video

In [10]:
#This function draws bbox on images
def draw_bbox(cv2, frame, bbox):
    
    top = bbox['top']
    left = bbox['left']
    bottom = bbox['bottom']
    right = bbox['right']
    
    cv2.rectangle(frame, (left, top), (right, bottom), (0, 165, 255), 10)
    
    return frame

### Load the video frames and render the bboxes

In [11]:
from IPython.display import display

import boto3
from boto3.dynamodb.conditions import Key

cap = cv2.VideoCapture(video_path)

frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

frame_id = 0

video_frames = []

# DynamoDB
table_name = 'octank_movie'

dynamodb = boto3.resource('dynamodb')

table = dynamodb.Table(table_name)

response = table.query(KeyConditionExpression=Key('video_id').eq(video_id))
                       
items = response['Items'][0]['detections']

product_ids = []
product_categories = []

while True:
    _, frame = cap.read()
    
    p_id = ''
    p_category = ''
    detections = items[str(frame_id)]
    for d in detections:
        frame = draw_bbox(cv2, frame, d['bbox'])
        p_id = d['product_id']
        p_category = d['product_category']
        
    
    frame_bytes = cv2.imencode('.jpeg', frame)[1].tobytes()

    video_frames.append(frame_bytes)
    product_ids.append(p_id)
    product_categories.append(p_category)
    
    frame_id += 1
    if frame_id >= frame_count:
        break

## Get User Info

In [14]:
user_id = 350

response = requests.get('http://{}/users/id/{}'.format(users_service_instance, user_id))
user = response.json()
persona = user['persona']

display(Markdown('Shopper persona for **user {}** is {}'.format(user_id, persona)))

Shopper persona for **user 350** is groceries_seasonal_tools

## Render Processed Video

In [31]:
# Get personalized recommendation for the user
item_list = get_recommendation(rfy_recommender_arn, 6, str(user_id))
display(Markdown(f'## Recommendation For User {user_id}: {persona}'))
evaluate_item_list(item_list, show=True)

box_layout = Layout(
                    display='flex',
                    flex_flow='column',
                    width='400%')

item_list = get_recommendation(wvx_recommender_arn, 4, str(user_id), product_ids[0])

recommend_box = recommended_item_layout(item_list, False)

vbox1 = VBox(children=[widgets.Image(value=video_frames[0])], layout=box_layout)

vbox2 = VBox(children=[widgets.Image(value=Image(product_image_url.format(product_categories[0],
                                                                          product_ids[0])).data),
                      widgets.Label(f"product_id: {product_ids[0]}"),
                      widgets.Label(f"product_category: {product_categories[0]}"),
                      widgets.Label("==== Freq Viewed Together ====="),
                      recommend_box])
        
def f(Frame_numb):
    i = Frame_numb-1
    image = video_frames[i]
    
    #Get recommendation of the products
    item_list = get_recommendation(wvx_recommender_arn, 4, str(user_id), product_ids[i])

    recommend_box = recommended_item_layout(item_list, False)
    
    vbox1 = VBox(children=[widgets.Image(value=image)], layout=box_layout)
    vbox2 = VBox(children=[widgets.Image(value=Image(product_image_url.format(product_categories[i],
                                                                          product_ids[i])).data),
                      widgets.Label(f"product_id: {product_ids[i]}"),
                      widgets.Label(f"product_category: {product_categories[i]}"),
                      widgets.Label("==== Freq Viewed Together ====="),
                      recommend_box])
    hbox = HBox([vbox1, vbox2])
    display(hbox)
    
hbox = HBox([vbox1, vbox2])

play = widgets.Play(
    value=1,
    min=1,
    max=frame_count,
    step=1,
    interval=2000,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider(min=1,max=frame_count,step=1,value=1)
widgets.jslink((play, 'value'), (slider, 'value'))
ui = widgets.HBox([play, slider])

output = widgets.interactive_output(f, {'Frame_numb': play})
display(ui, output)

## Recommendation For User 350: groceries_seasonal_tools

Output()

### Change User Personalized Recommendation

In [16]:
event_type_sample_set = {'AddToCart', 'Purchase', 'StartCheckout', 'View', 'ViewCart'}

properties_sample_set = {'{"discount": "No"}', '{"discount": "Yes"}'}

def generate_personalize_event(user_id, item_list, event_tracking_id):
    event =dict()
    event['trackingId'] = event_tracking_id
    event['userId'] = str(user_id)
    event['sessionId'] = str(uuid.uuid4())
    
    event_list = []
    for item in item_list:
        interaction = dict()
        interaction['eventId'] = str(uuid.uuid4())
        interaction['eventType'] = random.choice(tuple(event_type_sample_set))
        interaction['itemId'] = item['itemId']
        interaction['sentAt'] = int(time.time())
        interaction['properties'] = random.choice(tuple(properties_sample_set))
        
        event_list.append(interaction)
    
    event['eventList'] = event_list
    return event

In [18]:
item_list = [
    {'itemId': '988dde6a-b4a7-45a5-9e05-78dd796b6851'},
    {'itemId': '124db2fa-17c0-4e94-9844-d1b64a081df5'},
    {'itemId': '56dcfc2b-01d2-42d1-8002-32fdbe1a034a'},
]

evaluate_item_list(item_list, show=False)

In [30]:
for i in range(100):
    event = generate_personalize_event(user_id, item_list, event_tracking_id)

    response = kds.put_record(
        StreamName=stream_name,
        Data=json.dumps(event),
        PartitionKey=str(uuid.uuid4())
    )
    
    print(f'processed {i+1} interaction event for user {user_id}=======================')
    print(response['ResponseMetadata'])

processed 1 interaction event for user 350=======================
{'RequestId': 'fffedf8a-d570-08b6-a526-657d47084b36', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fffedf8a-d570-08b6-a526-657d47084b36', 'x-amz-id-2': 'cIFA1xt0nigJyPtQ/3zEutBqp/brcsZalRZn3kCX1uSgWMynthc9gD3OiDMZV5PY2pdHx530BqSWWHk5Eh6cjBHkwsAmwYfY', 'date': 'Tue, 19 Apr 2022 13:27:19 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 2 interaction event for user 350=======================
{'RequestId': 'd8b4b526-f776-f744-826c-0fd1650eb4c4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd8b4b526-f776-f744-826c-0fd1650eb4c4', 'x-amz-id-2': 'LOy1335KFpjdTG69CQKBdXHg7BEjSO9EBJ3kegwRM2eduFYk/ivu6pOI1gAgKUcCWBURTgo2XxwtPZKQURF1di1RtL5XL2lM', 'date': 'Tue, 19 Apr 2022 13:27:19 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 3 interaction event for user 350=======================
{'

processed 20 interaction event for user 350=======================
{'RequestId': 'd4d3c4cf-f19c-006f-8e0b-7e3763e443ef', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd4d3c4cf-f19c-006f-8e0b-7e3763e443ef', 'x-amz-id-2': 'EKDEFMmpsdOY2djwHtxL/Cavmxp2ecd+9MDQEEPTKlw8Degeu4bAogOcoHVnPPfxeTJGeM48GbKHj3goCv+uX0g67G2fM01v', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 21 interaction event for user 350=======================
{'RequestId': 'dfaf6f25-63be-94be-8577-d5ddf1c6d73e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dfaf6f25-63be-94be-8577-d5ddf1c6d73e', 'x-amz-id-2': 'R2OCMjfZK+UBC0n8FR+XS+AJ7THJnshPz3GWHEQToYUg6jtAIToP8axhGBSRjjM21hNjy4tjZQglSSbVTZnWQ63a58jqXwzv', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 22 interaction event for user 350=======================

processed 39 interaction event for user 350=======================
{'RequestId': 'd806a0c9-6359-e3e3-82de-1a31f121a063', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd806a0c9-6359-e3e3-82de-1a31f121a063', 'x-amz-id-2': 'LKGt7F2ies27ev11zloxxGhpEMomHWEjvFS/IdjTqhUtuGi0AffX/g0YHVlTEyS5aSbN5k9RO7us4+WmX/uTkUeP5LJjNHXh', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 40 interaction event for user 350=======================
{'RequestId': 'eab2b671-fd86-9ab0-b06a-0c896ffed930', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'eab2b671-fd86-9ab0-b06a-0c896ffed930', 'x-amz-id-2': 's41WjUQY1++71nlrv6I6+vEwjv27Msjrcl6TsYPg0N34w4C8i6PhJe0N3qzvCLSH1kgqkFQAQdEF3m0o7fFPu5ohEMPwc8sM', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 41 interaction event for user 350=======================

{'RequestId': 'dcc86bef-4b6a-3f28-8610-d117d9127ca8', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'dcc86bef-4b6a-3f28-8610-d117d9127ca8', 'x-amz-id-2': 'EUfocz8tG8t052vJnmWSuXe0T9SsDVXl1hk5TzacOVEJBJC/iGeP08rKO3/STWlke+O9BGlxFPrTYS0tBWK2SutJ94IzpAvb', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 58 interaction event for user 350=======================
{'RequestId': 'fe75d934-2b03-1f87-a4ad-63ccb97b5c07', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'fe75d934-2b03-1f87-a4ad-63ccb97b5c07', 'x-amz-id-2': 'wT72HYXq6GBhFLZgv2SryReaBykKc8a7ov1xt2LBs2sPFwtpWN4IXv7eNBZ8VUb2ZB5xP1KqbsCwBX3aLTxhjBgoFcElRfYA', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 59 interaction event for user 350=======================
{'RequestId': 'e81dbe0a-4a0c-10ac-b2c5-04f2d874532c', 'HTTPStatusC

processed 75 interaction event for user 350=======================
{'RequestId': 'd0c1f544-de76-2e66-8a19-4fbc4c0e6de6', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'd0c1f544-de76-2e66-8a19-4fbc4c0e6de6', 'x-amz-id-2': 'TO2VGy6EuAyDa4MThb9NNszpXjateR5bfVjfevd+4oVp6HhyId9Me2BYKkalyVpLPWmZ+kyuRehH5QVhrXhMQP51P33rnXXZ', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 76 interaction event for user 350=======================
{'RequestId': 'e3483e29-2691-fe7e-b990-84d1b4e9bdfe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e3483e29-2691-fe7e-b990-84d1b4e9bdfe', 'x-amz-id-2': 'RcyQUXPhMcM46jPi3GeRGTz7lvDTeHVzbSNmHwxW3xL4vY1FNXAH2xSnAIP5THRb+lLO9TaKWqEx9X2rRxrVoeNjk5qncPM5', 'date': 'Tue, 19 Apr 2022 13:27:20 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}
processed 77 interaction event for user 350=======================